## Sobol Financial Examples

In this notebook we demonstrate methods for Asian and European option pricing. Results are producing using both random numbers and Sobol sequences, paired with either Brownian Bridge or Standard approximations.

We're pricing 2 types of call option (European and Asian), using 2 techniques
* Black Scholes
* Monte Carlo

The Black Scholes functions
* bsEuroCall
* bsAsiaCall
give a closed-form solution, with a deterministic result.
Only dependence is the CumulativeNormal function from NormalDistribution.

The Monte Carlo functions
* mcEuroCall
* mcAsiaCall
each perform a number (nPaths) of random experiments.

For each experiment, we
1) Generate a number (nSteps) of random numbers (Uniform or Sobol)
2) Convert into N(0,1) random numbers (using InverseCumulativeNormal)
3) Convert into a Wiener path random walk (using standard or Brownian bridge method)
4) Convert into an asset price path, based on params (s,v,r,q,t)
5) Convert into an option price, based on option type and strike price (k)
Then average the prices to get a final predicted price

For each option type, we generate
* A single black scholes price
* A number (nTrials) of monte carlo prices
We can then take the RMSE of the MC prices, versus the BS price, to get an error.

We repeat this, for increasing values of nPaths, and compare the resulting errors.

---

In [1]:
/ load in q scripts
\l ../code/MonteCarlo.q
\l ../code/BlackScholes.q

In [2]:
/ float variables
/ * s = Asset at t=0
/ * k = Strike
/ * v = Volatility
/ * r = Interest rate, if Asian > .1
/ * q = Dividents
/ * t = Expiry
params:`s`k`v`r`q`t`mkt!(100 100 .2 .05 0 1.),`

/ integer variables
nSamplesStart:8      / initial number of samples
nSamplesTotal:128    / total number of paths
nsteps :1000         / number of time steps (per sample path)
ntrials:20           / number of trials (for RMSE)

/ booleans
/ * sobol   = SobolSeq(1), Random(0)
/ * bbridge = BrownianBridge(1), Standard(0)
bools:([]sobol:110b;bbridge:101b)

In [13]:
-1"European call";
params:params,enlist[`mkt]!enlist`european;
t:.z.t;
bs:bsEuroCall . params`s`k`v`r`q`t;
-1"Black-Scholes price = ",string bs;
-1"No. paths, Conv Err. Sobol/B. Bridge, Conv Err. Sobol/Std, Conv Err. Random/B. Bridge, Last Option Price";
N:nSamplesStart;
mcTrials[;params;nsteps;N;ntrials]each bools;
r:mcComparison[bs]each mcTrials[;params;nsteps;N;ntrials]each bools;
-1" , "sv enlist[string N],string[r[;1]],enlist string last r[;0];

European call
Black-Scholes price = 10.45058
No. paths, Conv Err. Sobol/B. Bridge, Conv Err. Sobol/Std, Conv Err. Random/B. Bridge, Last Option Price
8 , 10.45058 , 10.45058 , 10.45058 , 0


In [15]:
-1"Asian call";
params:params,enlist[`mkt]!enlist`asian;
t:.z.t;
bs:bsAsiaCall[nsteps] . params`s`k`v`r`q`t;
-1"Black-Scholes price = ",string bs;
-1"No. paths, Conv Err. Sobol/B. Bridge, Conv Err. Sobol/Std, Conv Err. Random/B. Bridge, Last Option Price";
N:nSamplesStart;
r:mcComparison[bs]each mcTrials[;params;nsteps;N;ntrials]each bools;
-1" , "sv enlist[string N],string[r[;1]],enlist string last r[;0];

Asian call
Black-Scholes price = 5.55152
No. paths, Conv Err. Sobol/B. Bridge, Conv Err. Sobol/Std, Conv Err. Random/B. Bridge, Last Option Price
8 , 8.509821 , 5.55152 , 14.60206 , 1.628969


In [16]:
mainfnc:{[p;nstart;ntot;nsteps;ntrials;bools]
 -1$[b:`asian~p`mkt;"Asian call";"European call"];t:.z.t;
 bs:$[b;bsAsiaCall nsteps;bsEuroCall]. params`s`k`v`r`q`t;
 -1"No. paths, Conv Err. Sobol/B. Bridge, Conv Err. Sobol/Std, Conv Err. Random/B. Bridge, Last Option Price";
 N:nstart;
 while[N<ntot;
  r:mcComparison[bs]each mcTrials[;params;nsteps;N;ntrials]each bools;
  -1" , "sv enlist[string N],string[r[;1]],enlist string last r[;0];
  N+:64];
 -1"CPU time = ",string[.z.t-t],"\n";}

In [17]:
peuro :params,enlist[`mkt]!enlist`european
pasian:params,enlist[`mkt]!enlist`asian
mainfnc[;nSamplesStart;nSamplesTotal;nsteps;ntrials;bools]each(peuro;pasian);

European call
No. paths, Conv Err. Sobol/B. Bridge, Conv Err. Sobol/Std, Conv Err. Random/B. Bridge, Last Option Price
8 , 6.29553 , 10.45058 , 8.90528 , 1.405293
72 , 6.29553 , 10.45058 , 10.03049 , 1.466703
CPU time = 00:00:45.857

Asian call
No. paths, Conv Err. Sobol/B. Bridge, Conv Err. Sobol/Std, Conv Err. Random/B. Bridge, Last Option Price
8 , 8.509821 , 5.55152 , 12.46473 , 1.397068
72 , 8.509821 , 5.55152 , 13.01684 , 1.473137
CPU time = 00:00:46.349



---